# Проект для «Викишоп»

## Введение

В рамках запуска нового сервиса сайта "Викишоп", позволяющего пользователям добавлять и редактировать описание товаров и услуг, необходимо разработать модель машинного обучения, которая будет определять тональность оставленного комментария и, в случае, если комментарий будет классифицирован, как негативный, отправлять его на дополнительную модерацию. Это позволит значительно разгрузить модераторов, которым необходимо будет валидировать только часть описаний. 
В нашем распоряжении ~ 150 000 тысяч комментариев с сайта викишоп, размеченные по уровню токсичности. 

**План работ:** 

1. Изучить данные. 
2. Лемматизировать тексты в корпусе. 
3. Выполнить токенизацию текстов корпуса, с оценкоц важности слов метрикой TF и IDF. 
4. Обучить модель машинного обчения и проверить ее на валидационной выборке. 

## Предобработка данных

In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import tqdm
from tqdm import notebook
nltk.download('punkt')
import re
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<div class="alert alert-block alert-info">
<b>Совет: </b> Желательно чтобы все импорты были собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Да, стараюсь собирать их в одном месте. Просто здесь как будто два проекта в одной тетрадке, поэтому импорты для Bert ниже. 
</div>

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df = df.drop(columns = 'Unnamed: 0')
#df = df[0:40000]

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Во всех колонках одинаковое количество строк, типы данных в колонках корректные.

## Подготовка данных к обучению

Создадим корпус текстов. 

In [4]:
corpus = df['text'].values

Выполним в цикле лемматизацию корпуса с заменой всех символов, отличных от латинского алфавита. 

In [ ]:
wnl = WordNetLemmatizer()
new_corpus = []
for i in notebook.tqdm(range(corpus.shape[0])):
    tokenized = nltk.word_tokenize(corpus[i])
    lemma = ' '.join([wnl.lemmatize(x) for x in tokenized])
    lemma = ' '.join(re.sub(r'[^a-zA-z]', ' ',lemma).split())
    new_corpus.append(lemma)
new_corpus = np.array(new_corpus).astype('U')

  0%|          | 0/159292 [00:00<?, ?it/s]

Лемматизация тестов выполнена. 

Создадим вектор для каждого текста с помощью метрик TF, IDF. 

Также учтем стоп-слова при оценке важности слов корпуса.

<div class="alert alert-block alert-success">
<b>Успех:</b> Очистка и лемматизация были сделаны правильно, молодец!
</div>

In [ ]:
target = df['toxic']
features_train, features_valid, target_train, target_valid = train_test_split(new_corpus, target, test_size=0.1, random_state=12345)
print (features_train.shape)
print (features_valid.shape)
print (target_train.shape)
print (target_valid.shape)

In [ ]:
stop_words = stopwords.words('english')
tf_idf_vect = TfidfVectorizer(stop_words=stop_words)
tf_idf_train = tf_idf_vect.fit_transform(features_train)
tf_idf_valid = tf_idf_vect.transform(features_valid)

Данные готовы для обучения модели. В качестве метрики будем использовать f1. Рассчет метрики осуществим с помощью кросс валидации. 

В качестве классификатора будем использовать CatBoost.

<div class="alert alert-block alert-success">
<b>Успех:</b> Разбиение было сделано верно. Отлично, что векторизатор был обучен только на тренировочной части данных.
</div>

## Обучение моделей

In [ ]:
model = CatBoostClassifier(iterations=4,
                           depth=5,
                           learning_rate=0.7,
                           loss_function='Logloss',
                           verbose=True)
model.fit(tf_idf_train, target_train)
scores = cross_val_score(model, tf_idf_train, target_train, cv=5, scoring = 'f1')
np.median(np.array(scores))

Также, попробуем обучить и оценить метрикой f1 с помощью кросс-валидации логистическую регрессию. 

In [ ]:
model = LogisticRegression(random_state=12345, max_iter=1200, class_weight='balanced', solver = 'lbfgs')
model.fit(tf_idf_train, target_train)
scores = cross_val_score(model, tf_idf_train, target_train, cv=5, scoring = 'f1')
np.median(np.array(scores))

## Валидация

In [ ]:
predictions = model.predict(tf_idf_valid)
score = round(f1_score(target_valid, predictions),2)
print ('метрика f1 составляет', score)

# Выводы

Для разработки модели анализа комментариев пользователей сайта "Викишоп" был использован корпус комментариев, размеченный по уровню негативности в размере 159 000 штук. 

Все тексты были лемматизированы, и токенизированы в рамках подготовки к обучению. 
В качестве моделей были выбраны Логистическая регрессия и catBoost. В качестве метрики качества была использована метрика f1. 

На валидационной выборке выбранная и обученная модель логистической регрессии метрика составила 0,75. 

<div class="alert alert-block alert-success">
<b>Успех:</b> Тестирование было сделано правильно.
</div>

# Проект для «Викишоп». BERT.  Факультативно.
#### Если возможно, прошу тажке оставить комментарии по реализации с BERT (у меня не получилось прогнать скрипты на полных данных, так как прогресс бар показывал 52 часа на создание эмбедингов, но интересно, насколько реализация близка к правде) Спасибо!

In [ ]:
import pandas as pd
import torch
import transformers
import numpy as np
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

In [ ]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.info()

Удаляю тексты, длиной более 512, так как модель не работает с текстами бОльшей длины. 

In [ ]:
df = df[0:400]
df = df.drop(columns = 'Unnamed: 0')
df = df.drop(index = df[df['text'].apply(lambda x: len(x))>512].index).reset_index(drop=True)

Токенизируем тексты

In [ ]:
tokenizer = transformers.BertTokenizer(vocab_file='vocab.txt')

In [ ]:
notebook.tqdm.pandas()
tokenized = df['text'].progress_apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

Проверим максимальную длину. 

In [ ]:
max_len = 0 
for i in range(len(tokenized)):
    if len(tokenized[i])>max_len:
        max_len = len(tokenized[i])
        max_len_token = i
max_len

Уравняем длины полчившихся, после токенизации векторов. 
Также, удалим лишние нули из векторов, сохраняя единую длину. 

In [ ]:
padded = []
for i in range(len(tokenized)):
    padded_i = tokenized[i] + [0]*(max_len-len(tokenized[i]))
    padded.append(padded_i)
padded = np.array(padded)
attention_mask = np.where(padded!=0, 1, 0)

In [ ]:
padded.shape

Создадим эмбединги

In [ ]:
config = transformers.BertConfig.from_json_file('bert_config.json')
model = transformers.BertModel.from_pretrained('pytorch_model.bin', config = config)

In [ ]:
embendings = []
batch_size = 100

for i in notebook.tqdm(range(padded.shape[0]//batch_size + 1)):
    batch = torch.LongTensor(padded[i*batch_size:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    
    with torch.no_grad():
        batch_embendings = model(batch, attention_mask = attention_mask_batch)
    
    embendings.append(batch_embendings[0][:,0,:].numpy())

In [ ]:
features = np.concatenate(embendings)

## Обучение

In [ ]:
target = df['toxic']
features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.2, 
                                                                              random_state=12345)

In [ ]:
model = CatBoostClassifier(iterations=2,
                           depth=2,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)
model.fit(features_train, target_train)
scores = cross_val_score(model, features_train, target_train, cv = 5, scoring='f1')
np.median(np.array(scores))

In [ ]:
model = LogisticRegression(random_state=12345, max_iter=1200)
model.fit(features_train, target_train)
scores = cross_val_score(model, features_train, target_train, cv = 5, scoring='f1')
np.median(np.array(scores))

## Валидация

In [ ]:
predictions = model.predict(features_valid)
score = round(f1_score(target_valid, predictions),2)
print ('метрика f1 составляет', score)